In [2]:
import importlib
import pandas as pd #type: ignore
import numpy as np #type: ignore
import scipy.stats as stats #type: ignore
import sys, os
import datetime as dt

import pricer #type: ignore
importlib.reload(pricer)

pd.options.display.float_format = '{:,.8f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [3]:
coinbaseexchange_creds = {
        'apiKey' : "ADD YOURS HERE",
        'secret' : "ADD YOURS HERE",
        'password': "ADD YOURS HERE",
    }

In [4]:
# The pricer lib is a powerful class that I coded for the 1st version of the pricer.
# It solely focus on fetching market data. You write the exchange you want to fetch data, 
# the spot pair and perp pair (if you want to fetch for perp as well, otherwise you don't need it)
# The class will immediately fetch current snapshot of orderbook, ohlcv data for spot and perp (with the time range
# that you can modify or leave it as default, check the class for default values), spot and perp mid, top bid and ask for
# spot and perps, historical funding rate and predicted next funding rate
# You need to instantiate 1 class per exchange and per pair

# Below I'm instantiating the class for Binance and for SUI, and increasing the interval fetched for "1m" candles
# Numbers on modified_date_range are hours. i.e. for "1m" we are fetching 96h

modified_date_range = {"1s": 1, "1m": 96, "5m": 1, "1h": 2160, "1d": 8760}
binance_market_data = pricer.MarketData("binance", "SUI/USDT", "SUI/USDT:USDT", date_range = modified_date_range)
coinbase_market_data = pricer.MarketData("coinbaseexchange", "USDT/USD", date_range = modified_date_range, credentials=coinbaseexchange_creds)

ExchangeNotAvailable: binance GET https://api.binance.com/api/v3/exchangeInfo 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}

In [14]:
# After the class is istantiated, it automatically creates several class attributes:
# ob_data_spot: pd.DataFrame -> has the snapshot of the spot pair orderbook
# ob_data_future: pd.DataFrame -> has the snapshot of the perp pair orderbook
# ohlcv_data: dict -> Dictionary with 2 keys, 'spot' and 'perp', which are also dictionaries.
# Each one of 'spot' and 'perp' have keys on the interval, e.g. "1m"
# spot_mid_price: float -> current spot mid price
# spot_mid_price: float -> current perp mid price
# top_bid_ask: dict -> Dictionary with 2 keys, 'spot' and 'perp', with the top bid ask prices for spot and perp
# historical_funding_rate: pd.DataFrame -> last 30 funding rates
# funding_rate_summary: dict -> Dictionary with statistics about the funding rate (e.g. mean, std, min, max)
# next_funding_rate: float -> Predicted next funding rate
# funding_datetime: str -> String representation of the next funding rate date (e.g. '2024-07-26T16:00:00.000Z')
# next_funding_seconds: float -> how many seconds until next funding payment

# To fetch a call attribute, you can do like (and using the binance_market_data as above):
# binance_market_data.ohlcv_data["spot"]["1m"] 
# binance_market_data.historical_funding_rate

In [72]:
#Order details
order_size = 8000000
order_duration_h = 24
expected_fill_per_minute = order_size / (order_duration_h * 60)
order_start_time = "2024-07-22 13:00:00"
order_end_time = (pd.to_datetime(order_start_time)+dt.timedelta(hours=order_duration_h)).strftime("%Y-%m-%d %H:%M:%S")
direction = "sell"
galaxy_fees = 18 #in bps
multiplier = 1 if direction=="buy" else -1

In [75]:
# Copying the dataframe so we can modify it as we want without affecting the class
sui_1m_ohlcv = binance_market_data.ohlcv_data["spot"]["1m"].copy()
usdt_1m_ohlcv = coinbase_market_data.ohlcv_data["spot"]["1m"].copy()

# Using the same dataframe as galaxy provided in the xlsv
sui_1m_ohlcv = sui_1m_ohlcv.loc[order_start_time:order_end_time]
usdt_1m_ohlcv = usdt_1m_ohlcv.loc[order_start_time:order_end_time]
sui_1m_ohlcv["above_limit"] = np.where(sui_1m_ohlcv["close"]>=0.8515, True, False)
sui_close_above_limit_df = sui_1m_ohlcv.query("above_limit == True")[["close"]].copy()
n_close_above_limit = len(sui_close_above_limit_df)
usdt_close_above_limit_df = usdt_1m_ohlcv.merge(sui_close_above_limit_df, left_index=True, right_index=True, suffixes=("", "_sui"))

In [79]:
# round(sui_1m_ohlcv["above_limit"].mean(),5)
sui_avg_price = round(sui_close_above_limit_df["close"].mean(),9)
usdt_avg_price = round(usdt_close_above_limit_df["close"].mean(),9)
final_price = round(sui_avg_price * usdt_avg_price,9)
net_final_price = round(final_price*(1+(multiplier*galaxy_fees)/10000),9)
print(f"SUI/USDT Average: {sui_avg_price}\nUSDT/USD Average: {usdt_avg_price}\n\
SUI/USD(C) Average: {final_price}\nNet Price: {net_final_price}\nExpected Fills: {n_close_above_limit*expected_fill_per_minute}")

SUI/USDT Average: 0.85218
USDT/USD Average: 1.000267
SUI/USD(C) Average: 0.852407532
Net Price: 0.850873198
Expected Fills: 111111.11111111111
